In [380]:
'''Plot qq plot to compare distribution of LogLR values
with the chisq (df=1) distribution.'''

%pylab inline
import sys
sys.path.append("/storage/BonnieH/selection_project/helper_functions")
from LRT_functions import *
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
from matplotlib import pyplot as plt
from scipy import stats
import time
PLOTDIR = '/storage/BonnieH/selection_project/validation_per_locus/figures/qq_plot/'

Populating the interactive namespace from numpy and matplotlib


In [381]:
# Function to plot QQ plot
def plot_figure(fig_num, per, opt_allele_list, LogLR_vals_dic, p_vals_dic, abc_model):
    
    # Get minimum length of all LogLR lists
    min_length = 500
    for opt_allele in opt_allele_list:
        length = len(LogLR_vals_dic[opt_allele])
        if length < min_length:
            min_length = length
    
    for opt_allele in opt_allele_list:
        LogLR_list = LogLR_vals_dic[opt_allele]
        
        # Plot histogram of LogLR values
        fig_num = fig_num + 1
        plt.figure(fig_num)
        plt.hist(LogLR_list, bins = 20, weights=np.ones(len(LogLR_list)) / len(LogLR_list)) 
        plt.title('Plot of LogLR values \n Per %d Opt %d'%(per, opt_allele))
        plt.xlabel('LogLR value')
        plt.ylabel('Fraction of values')
        plt.savefig(PLOTDIR + 'LogLR/LogLR_per_%d_opt_%d'%(per,opt_allele), bbox_inches='tight')
        
        # Plot histogram of LogLR values where negative values are set to 0
        fig_num = fig_num + 1
        plt.figure(fig_num)
        LogLR_list_no_neg = [0 if i < 0 else i for i in LogLR_list] 
        LogLR_list_no_neg.sort()
        plt.hist(LogLR_list_no_neg, bins = 20, weights=np.ones(len(LogLR_list_no_neg)) / len(LogLR_list_no_neg)) 
        plt.title('Plot of LogLR values (setting negative values to 0) \n Per %d Opt %d'%(per, opt_allele))
        plt.xlabel('LogLR value')
        plt.ylabel('Fraction of values')
        plt.savefig(PLOTDIR + 'LogLR/LogLR_no_neg_per_%d_opt_%d'%(per,opt_allele), bbox_inches='tight')
        
    # Plot Chi-square distribution
    fig_num = fig_num + 1
    plt.figure(fig_num)
    
    exp_list = []
    for i in range(0, min_length):
        LogLR = np.random.chisquare(1)
        exp_list.append(float(LogLR))
    exp_list.sort()
    plt.hist(exp_list, bins = 20, weights=np.ones(len(exp_list)) / len(exp_list), color='green') #weights=np.ones(exp_list) / len(exp_list))
    plt.title('Plot of chisq (df=1) distribution')
    plt.savefig(PLOTDIR + 'Chisq/Chisq_for_per_%d'%(per), bbox_inches='tight')
        
    # Plot QQ plot LogLR v chisq distribution
    fig_num = fig_num + 1
    plt.figure(fig_num)
    x_max = max(exp_list)
    for opt_allele in opt_allele_list:
        LogLR_list = LogLR_vals_dic[opt_allele][0:min_length]
        LogLR_list_no_neg = [0 if i < 0 else i for i in LogLR_list] 
        LogLR_list_no_neg.sort()
        plt.scatter(exp_list, LogLR_list_no_neg, s=5, label=str(opt_allele)) 
    
        x_max_observed = max(LogLR_list_no_neg)
        if x_max_observed > x_max:
            x_max = x_max_observed
    plt.plot([0,x_max],[0,x_max],c='black')
    
    plt.title('LogLR v chisq (df=1) distribution \n Per %d'%(per))
    plt.xlabel("Expected LogLR (chisq df=1)")
    plt.ylabel("Observed LogLR")
    plt.legend()
    plt.savefig(PLOTDIR + 'qq_LogLR/qq_LogLR_per_%d'%(per), bbox_inches='tight')
     
    # Plot QQ plot of p values of LogLR v chisq distribution
    fig_num = fig_num + 1
    plt.figure(fig_num)
    
    p_val_exp = []
    for elem in exp_list:
        pval = chi2.sf(elem, 1) 
        p_val_exp.append(-np.log10(pval))
    x_max = max(p_val_exp)
    p_val_exp.sort()
    
    for opt_allele in opt_allele_list:
        p_val_obs = p_vals_dic[opt_allele][0:min_length]
        
        p_val_obs.sort()
        
        plt.scatter(p_val_exp, p_val_obs, s=5, label=str(opt_allele)) # color="black", label='coding')
    
        x_max_observed = max(p_val_obs)
       
        if x_max_observed > x_max:
            x_max = x_max_observed
    
    plt.plot( [0,x_max],[0, x_max],c='black' )
    plt.title('LogLR v chisq (df=1) -log10(p) distribution \n Per %d'%(per))
    plt.xlabel("Expected -log10(p) (chisq df=1)")
    plt.ylabel("Observed -log10(p)")
    plt.legend()
    plt.savefig(PLOTDIR + 'qq_p_val/qq_p_val_per_%d'%(per), bbox_inches='tight')

    print('Done figure ' + str(fig_num))
    

In [382]:
# Function to validate ABC and LRT
def validate(per, opt_allele, s_vals, use_het, use_common, use_bins, num_bins, abc_model, lrt_model, fig_num):
    
    # Process list of optimal alleles
    opt_allele_list = list(opt_allele.split(','))
    opt_allele_list = list(map(int, opt_allele_list)) 
    
    # Process list of s values
    s_vals = list(s_vals.split(','))
    s_vals = list(map(float, s_vals))
    
    # ABC parameters
    constant_het = 0.005
    denom_het = 3
    constant_common = 1
    denom_common = 4
    eps_bins = 0.3
    
    # LRT parameters
    LRT_num_sims = 200
         
    # Each dictionary contains values for all optimal alleles
    # Key: optimal allele
    # Value: list of mean values for each s 
    s_vals_dic = {}
    p_vals_dic = {}
    LogLR_vals_dic = {}
    
    # Initialize dictionaries above
    for opt_allele in opt_allele_list:
        s_vals_dic[opt_allele] = []
        p_vals_dic[opt_allele] = []
        LogLR_vals_dic[opt_allele] = []
      
    # Run ABC and LRT for each opt_allele
    for opt_allele in opt_allele_list:
        LogLR_list = []
        l0_list = []
        ls_list = []
        print('Running per: ' + str(per) + ' optimal allele: ' + str(opt_allele))
        no_ABC_accept = 0 # Number of s values with <10 ABC acceptances
        
        ### Get list of s values in LRT simulations file ###
        s_list_available = []
        lrtFile = '/gymreklab-tscc/bonnieh/lrt/results/' + lrt_model + '/' + str(per) + '_' + str(opt_allele) + '_freqs.txt' # euro_prelim #const_prelim
        lrt_file = open(lrtFile, 'r')
        header = lrt_file.readline().strip()
    
        for line in lrt_file:
            info = line.strip().split('\t')
            s = float(info[0])
            if s not in s_list_available:
                s_list_available.append(s)
            
        est_s_dic = {} # Dictionary of estimated s values; Key: True value of s, Value: list of estimated s values
        
        # Put summary statistics in dictionaries
        obs_het_dic = {} # Key - s; value - list of het
        obs_comm_dic = {} # Key - s; value - list of number of common alleles (frequency >= 5%)
        obs_bins_dic = {} # Key - s; value - list of bins (bins are given as lists)
        
        # Same as dictionaries above except without simulations 
        # where s could not be estimated using ABC (<10 ABC acceptances)
        # These are the simulations used for LRT
        obs_het_dic_lrt = {}
        obs_comm_dic_lrt = {}
        obs_bins_dic_lrt = {}
    
        # Fill in dictionaries of summary statistics
        for s in s_vals:
            
            obs_het_dic[s] = []
            obs_comm_dic[s] = []
            obs_bins_dic[s] = []
            
            freqs_list_raw = GetLRTListFreq(lrtFile, s) # Get list of allele frequencies for this s
            
            # Get summary statistics from frequencies
            for freq_string in freqs_list_raw:
                
                obs_het, obs_common, obs_bins = GetSummStats(freq_string, num_bins)
                obs_het_dic[s].append(obs_het)
                obs_comm_dic[s].append(obs_common)
                obs_bins_dic[s].append(obs_bins)
            
        abcFile = '/gymreklab-tscc/bonnieh/abc/results/' + abc_model +'/' + str(per) + '_' + str(opt_allele) + '.txt' 

        # Read abcFile line by line and place in lookup table in the form of a list
        abc_list = GetABCList(abcFile, num_bins)
        
        # Perform ABC
        for s in s_vals:
            
            list_est_s = [] # List of posterior estimates of s
            
            # Lists of summary statistics of s
            list_het = []
            list_common = []
            list_bins = []
            
            for i in range(0, len(obs_het_dic[s])):
                obs_het  = float(obs_het_dic[s][i])
                obs_common = int(obs_comm_dic[s][i])
                obs_bins = obs_bins_dic[s][i]
                
                s_ABC, lower_bound, upper_bound, num_accepted, s_accepted = Get_S_ABC(abc_list, \
                                       obs_het, obs_common, obs_bins, constant_het, \
                                       denom_het, constant_common, denom_common, eps_bins, use_het, \
                                       use_common, use_bins)
                if s_ABC != -1:
        
                    s_ABC_round = get_LRT_bin(s_ABC)
            
                    # Get nearest s in LRT file to perform LRT
                    if s_ABC_round not in s_list_available:
                        #print('Not available: per %d opt allele %d s_ABC_round %.5f'%(per, opt_allele, s_ABC_round))
                        s_ABC_round = getNearestS(s_ABC_round, s_list_available)
                        #print('Nearest s: %.5f'%(s_ABC_round))
                    list_est_s.append(s_ABC_round) 
                    list_het.append(obs_het)
                    list_common.append(obs_common)
                    list_bins.append(obs_bins)
                else:
                    no_ABC_accept = no_ABC_accept + 1
                    print(no_ABC_accept)
                    
            est_s_dic[s] = list_est_s
            obs_het_dic_lrt[s] = list_het
            obs_comm_dic_lrt[s] = list_common
            obs_bins_dic_lrt[s] = list_bins
            
            # Put mean of esimated s in s_vals_dic 
            s_vals_dic[opt_allele].append(np.mean(list_est_s)) 
            
        # Get LRT summary statistic tables for s = 0
        lrtFile_for_s_0 = '/gymreklab-tscc/bonnieh/lrt/results/' + lrt_model[:-6] + '/' + str(per) + '_' + str(opt_allele) + '_15_freqs.txt' # euro_prelim #const_prelim
        freqs_list_raw_0 = GetLRTListByRow(lrtFile_for_s_0, 0)
        #freqs_list_raw_0 = GetLRTListFreq(lrtFile, 0)
        LRT_table_0_het = []
        LRT_table_0_common = []
        LRT_table_0_bins = []
        
        # Get summary statistics from allele frequencies
        for freq_string in freqs_list_raw_0:
                
            obs_het, obs_common, obs_bins = GetSummStats(freq_string, num_bins)
            LRT_table_0_het.append(obs_het) 
            LRT_table_0_common.append(obs_common) 
            LRT_table_0_bins.append(obs_bins)
                
        # Perform LRT
        for s in est_s_dic:
            p_vals_list = []
            obs_het_list = obs_het_dic_lrt[s]
            obs_common_list = obs_comm_dic_lrt[s]
            obs_bins_list = obs_bins_dic_lrt[s]
            s_ABC_list = est_s_dic[s]
            
            for i in range(0, len(obs_het_list)):
                obs_het = obs_het_list[i]
                obs_common = obs_common_list[i]
                obs_bins = obs_bins_list[i]
                s_ABC_round = s_ABC_list[i]
                
                ### Use commented code if didn't round during ABC ###
                '''
                s_ABC_round = get_LRT_bin(s_ABC_round)
                if s_ABC_round not in s_list_available:
                    s_ABC_round = getNearestS(s_ABC_round, s_list_available)
                '''
                
                # Get LRT summary statistic tables for s = s_ABC_round
                if s_ABC_round == 0:
                    freqs_list_raw_s = GetLRTListByRow(lrtFile_for_s_0, 1)
                else:
                    freqs_list_raw_s = GetLRTListFreq(lrtFile, s_ABC_round)
            
                LRT_table_s_het = []
                LRT_table_s_common = []
                LRT_table_s_bins = []
                
                # Get summary statistics from allele frequencies
                for freq_string in freqs_list_raw_s:
                    
                    obs_het_s_ABC, obs_common_s_ABC, obs_bins_s_ABC = GetSummStats(freq_string, num_bins)
                    LRT_table_s_het.append(obs_het_s_ABC) 
                    LRT_table_s_common.append(obs_common_s_ABC) 
                    LRT_table_s_bins.append(obs_bins_s_ABC)
                
                if len(LRT_table_s_het) != 0:
                    likelihood_0, likelihood_s_ABC, LR, LogLR, pval = LikelihoodRatioTest(LRT_table_0_het, LRT_table_s_het, \
                                LRT_table_0_common, LRT_table_s_common, LRT_table_0_bins, LRT_table_s_bins, LRT_num_sims, \
                                obs_het, obs_common, obs_bins, constant_het, denom_het, \
                                constant_common, denom_common, eps_bins, use_het, use_common, use_bins)
                
                    p_vals_list.append(-np.log10(pval))
                    LogLR_list.append(LogLR)
                    l0_list.append(likelihood_0)
                    ls_list.append(likelihood_s_ABC)
        
            LogLR_vals_dic[opt_allele] = LogLR_list
            
            p_vals_dic[opt_allele] = p_vals_list
          
    # Plot QQ Plot
    fig_num = fig_num + 1
    plot_figure(fig_num, per, opt_allele_list, LogLR_vals_dic, p_vals_dic, abc_model)
    #fig_num = fig_num + 1
    #plot_figure(fig_num, per, opt_allele_list, p_vals_dic, errors_p_dic, s_vals, use_het, use_common, use_bins, num_bins, eps_bins, abc_model, 'lrt')
    return fig_num

In [383]:
def main():
    print('Running main')
    
    # List of periods to show distribution of LogLR 
    per_list = [4] 
    
    # Dictionary of optimal alleles 
    # Key: period, Value: string of optimal alleles to validate separated by commas
    opt_alleles = {}
    opt_alleles[1] = '16,24,32,40'
    opt_alleles[2] = '11,14,17,20'
    opt_alleles[3] = '6,8,10,12'
    #opt_alleles[3] = '5,6,7,8,9,10,11,12'
    opt_alleles[4] = '7,8,9,10'
    
    s_vals = '0'
    
    # Summary statistics to use - 'y' = yes, 'n' = no
    use_het = 'y'
    use_common = 'n'
    use_bin = 'y'
    
    #num_bins_list = [3,5,7] # Number of bins to use for validation
    num_bins_list = [5]
    
    # Priors to use 
    model_list = [('eurodem_p2','eurodem_merge')] 
    fig_num = 0
    
    # Run validation
    for per in per_list:
        for num_bins in num_bins_list:
            for model in model_list:
                fig_num = validate(per, opt_alleles[per], s_vals, use_het, use_common, use_bin, num_bins, model[0], model[1], fig_num)

In [ ]:
%%time
if __name__ == '__main__':
    main()

Running main
Running per: 4 optimal allele: 7
1
